# **Claiming Bonus marks for chatgpt for 200 sentences.My name is komal**

# PROBLEM 1

In [ ]:
!pip install transformers

In [ ]:
import random

def select_and_write_same_random_samples(file_paths, num_samples, output_file_paths):
    samples_indices = random.sample(range(num_samples), num_samples)
    samples = []

    for file_path in file_paths:
        with open(file_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            samples.append([lines[i].strip() for i in samples_indices])

    for samples, output_file_path in zip(samples, output_file_paths):
        with open(output_file_path, "w", encoding="utf-8") as output_file:
            for sample in samples:
                output_file.write(sample + "\n")

# Example usage
file_paths = ["test.te", "test.hi", "test.en"]
num_samples = 1000
output_file_paths = ["tests.te", "tests.hi", "tests.en"]
select_and_write_same_random_samples(file_paths, num_samples, output_file_paths)


**nllb**

In [15]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_name="facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
def translate_text(article, target_lang_code):
    inputs = tokenizer(article, return_tensors="pt")

    target_lang_id = tokenizer.lang_code_to_id.get(target_lang_code, None)
    if target_lang_id is None:
        raise ValueError(f"Invalid target language code: {target_lang_code}")

    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=target_lang_id, max_length=30
    )
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translated_text


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [16]:
import json

def translate_lines(input_file, output_file, target_language):
    with open(input_file, "r", encoding="utf-8") as file:
        lines = file.readlines()

    with open(output_file, 'w', encoding='utf-8') as f:
        for line in lines:
            line = line.strip()
            translation = translate_text(line, target_language)
            f.write(translation + '\n')

    print(f"Translations written to {output_file}")


In [17]:
eng_code="eng_Latn"
hin_code="hin_Deva"
tel_code="tel_Telu"

In [ ]:
translate_lines("tests.en", "en_te.nllb", tel_code)

In [ ]:
translate_lines("tests.te", "te_en.nllb", eng_code)

In [ ]:
translate_lines("tests.hi", "hi_te.nllb", tel_code)

In [18]:
translate_lines("tests.te", "te_hi.nllb", hin_code)

the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38


Translations written to te_hi.nllb


In [ ]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [ ]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [ ]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./
%cd ..

# **Restart session**

**IndicTrans**

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [ ]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [ ]:
def write_to_file(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        f.writelines(data)

**English to telugu**

In [ ]:
ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer, model = initialize_model_and_tokenizer(ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)
input_filename = "tests.en"
with open(input_filename, "r", encoding="utf-8") as f:
    sents = f.readlines()

# Translate sentences
src_lang, tgt_lang ="eng_Latn","tel_Telu"
translations = batch_translate(sents, src_lang, tgt_lang, model, tokenizer, ip)

# Write input sentences and their translations to an output file
output_filename = "en_te.indictrans"
output_data = []
for translation in translations:
    output_data.append(translation+"\n")
write_to_file(output_filename, output_data)
del tokenizer, model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

**Telugu to English**

In [ ]:
ckpt_dir = "ai4bharat/indictrans2-indic-en-dist-200M"
tokenizer, model = initialize_model_and_tokenizer(ckpt_dir, "indic-en", "")

ip = IndicProcessor(inference=True)
input_filename = "tests.te"
with open(input_filename, "r", encoding="utf-8") as f:
    sents = f.readlines()

# Translate sentences
src_lang, tgt_lang ="tel_Telu","eng_Latn"
translations = batch_translate(sents, src_lang, tgt_lang, model, tokenizer, ip)

# Write input sentences and their translations to an output file
output_filename = "te_en.indictrans"
output_data = []
for translation in translations:
    output_data.append(translation+"\n")
write_to_file(output_filename, output_data)
del tokenizer, model

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/914M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

**Telugu to Hindi**

In [5]:
ckpt_dir = "ai4bharat/indictrans2-indic-indic-dist-320M"
tokenizer, model = initialize_model_and_tokenizer(ckpt_dir, "indic-indic", quantization)

ip = IndicProcessor(inference=True)
input_filename = "tests.te"
with open(input_filename, "r", encoding="utf-8") as f:
    sents = f.readlines()

# Translate sentences
src_lang, tgt_lang ="tel_Telu","hin_Deva"
translations = batch_translate(sents, src_lang, tgt_lang, model, tokenizer, ip)

# Write input sentences and their translations to an output file
output_filename = "te_hi.indictrans"
output_data = []
for translation in translations:
    output_data.append(translation+"\n")
write_to_file(output_filename, output_data)
del tokenizer, model

config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

**Hindi to Telugu**

In [6]:
ckpt_dir = "ai4bharat/indictrans2-indic-indic-dist-320M"
tokenizer, model = initialize_model_and_tokenizer(ckpt_dir, "indic-indic", quantization)

input_filename = "tests.hi"
with open(input_filename, "r", encoding="utf-8") as f:
    sents = f.readlines()

# Translate sentences
src_lang, tgt_lang ="hin_Deva","tel_Telu"
translations = batch_translate(sents, src_lang, tgt_lang, model, tokenizer, ip)

# Write input sentences and their translations to an output file
output_filename = "hi_te.indictrans"
output_data = []
for translation in translations:
    output_data.append(translation+"\n")
write_to_file(output_filename, output_data)
del tokenizer, model

# PROBLEM 2

In [7]:
!pip install sacrebleu

In [8]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d2fcacbcaf3e84b916363a9381b1013c0d23721e15d6c7667508e8bfbd9e9d3a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [9]:
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer

def calculate_scores(reference_file, translation_file):
    # Load reference and translation files
    with open(reference_file, 'r', encoding='utf-8') as f:
        references = [line.strip() for line in f]
    with open(translation_file, 'r', encoding='utf-8') as f:
        translations = [line.strip() for line in f]

    # Calculate BLEU score
    bleu = corpus_bleu(translations, [references], lowercase=True).score

    # Calculate ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(' '.join(translations), ' '.join(references))
    print("Bleuscore:", bleu)
    print("RougeScores:")
    print("rouge1:",scores['rouge1'])
    print("rouge2:",scores['rouge2'])
    print("rougeL:",scores['rougeL'])


**NLLB**

**Telugu to English**

In [19]:
calculate_scores("tests.en", "te_en.nllb")

Bleuscore: 21.813093036675138
RougeScores:
rouge1: Score(precision=0.8001345483456669, recall=0.8010163472723933, fmeasure=0.8005752049932688)
rouge2: Score(precision=0.44024464831804283, recall=0.44072985549840804, fmeasure=0.44048711829141424)
rougeL: Score(precision=0.4734878600697205, recall=0.4740096736668095, fmeasure=0.47374862317953736)


**English to Telugu**

In [20]:
calculate_scores("tests.te", "en_te.nllb")

Bleuscore: 4.55029864603341
RougeScores:
rouge1: Score(precision=0.8556149732620321, recall=0.8247422680412371, fmeasure=0.8398950131233595)
rouge2: Score(precision=0.5268817204301075, recall=0.5077720207253886, fmeasure=0.5171503957783641)
rougeL: Score(precision=0.7807486631016043, recall=0.7525773195876289, fmeasure=0.7664041994750657)


**Telugu to Hindi**

In [35]:
calculate_scores("tests.hi", "te_hi.nllb")

Bleuscore: 15.378225063193861
RougeScores:
rouge1: Score(precision=0.40145985401459855, recall=0.9217877094972067, fmeasure=0.559322033898305)
rouge2: Score(precision=0.18048780487804877, recall=0.4157303370786517, fmeasure=0.2517006802721088)
rougeL: Score(precision=0.36009732360097324, recall=0.8268156424581006, fmeasure=0.5016949152542374)


**Hindi to Telugu**

In [22]:
calculate_scores("tests.te", "hi_te.nllb")

Bleuscore: 4.728873542522401
RougeScores:
rouge1: Score(precision=0.8502673796791443, recall=0.8071065989847716, fmeasure=0.828125)
rouge2: Score(precision=0.478494623655914, recall=0.45408163265306123, fmeasure=0.46596858638743455)
rougeL: Score(precision=0.7593582887700535, recall=0.7208121827411168, fmeasure=0.7395833333333335)


**IndicTrans**

**Telugu to English**

In [23]:
calculate_scores("tests.en", "te_en.indictrans")

Bleuscore: 35.92421993551058
RougeScores:
rouge1: Score(precision=0.8908323649929668, recall=0.8679537599809319, fmeasure=0.8792442579905231)
rouge2: Score(precision=0.5640978593272171, recall=0.5496096776115845, fmeasure=0.5567595303492198)
rougeL: Score(precision=0.626200232401688, recall=0.6101179835538076, fmeasure=0.6180545075906196)


**English to Telugu**

In [24]:
calculate_scores("tests.te", "en_te.indictrans")

Bleuscore: 6.648508481753942
RougeScores:
rouge1: Score(precision=0.9037433155080213, recall=0.8407960199004975, fmeasure=0.8711340206185567)
rouge2: Score(precision=0.6182795698924731, recall=0.575, fmeasure=0.5958549222797926)
rougeL: Score(precision=0.8342245989304813, recall=0.7761194029850746, fmeasure=0.8041237113402062)


**Telugu to Hindi**

In [25]:
calculate_scores("tests.hi", "te_hi.indictrans")

Bleuscore: 19.56360624591407
RougeScores:
rouge1: Score(precision=0.41849148418491483, recall=0.9297297297297298, fmeasure=0.5771812080536913)
rouge2: Score(precision=0.1951219512195122, recall=0.43478260869565216, fmeasure=0.26936026936026936)
rougeL: Score(precision=0.3746958637469586, recall=0.8324324324324325, fmeasure=0.5167785234899328)


**Hindi to Telugu**

In [26]:
calculate_scores("tests.te", "hi_te.indictrans")

Bleuscore: 4.622220388102432
RougeScores:
rouge1: Score(precision=0.893048128342246, recall=0.755656108597285, fmeasure=0.8186274509803922)
rouge2: Score(precision=0.532258064516129, recall=0.45, fmeasure=0.4876847290640394)
rougeL: Score(precision=0.7914438502673797, recall=0.669683257918552, fmeasure=0.7254901960784313)


**Chatgpt**

**Telugu to English**

In [30]:
calculate_scores("test200.en", "te_en.chatgpt")

Bleuscore: 15.603030443481524
RougeScores:
rouge1: Score(precision=0.7765196662693683, recall=0.6940079893475366, fmeasure=0.7329489523273801)
rouge2: Score(precision=0.35827123695976154, recall=0.32019179541822057, fmeasure=0.33816289210859474)
rougeL: Score(precision=0.44845053635280097, recall=0.4007989347536618, fmeasure=0.42328786387287304)


**English to Telugu**

In [31]:
calculate_scores("test200.te", "en_te.chatgpt")

Bleuscore: 1.9851886775021763
RougeScores:
rouge1: Score(precision=0.8076923076923077, recall=0.7, fmeasure=0.75)
rouge2: Score(precision=0.36, recall=0.3103448275862069, fmeasure=0.3333333333333333)
rougeL: Score(precision=0.7307692307692307, recall=0.6333333333333333, fmeasure=0.6785714285714285)


**Telugu to Hindi**

In [32]:
calculate_scores("test200.hi", "te_hi.chatgpt")

Bleuscore: 7.862774986327221
RougeScores:
rouge1: Score(precision=0.3392857142857143, recall=0.8636363636363636, fmeasure=0.48717948717948717)
rouge2: Score(precision=0.14545454545454545, recall=0.38095238095238093, fmeasure=0.21052631578947364)
rougeL: Score(precision=0.30357142857142855, recall=0.7727272727272727, fmeasure=0.43589743589743585)


**Hindi to Telugu**

In [33]:
calculate_scores("test200.te", "hi_te.chatgpt")

Bleuscore: 1.7493397518500644
RougeScores:
rouge1: Score(precision=0.8076923076923077, recall=0.7777777777777778, fmeasure=0.7924528301886792)
rouge2: Score(precision=0.36, recall=0.34615384615384615, fmeasure=0.35294117647058826)
rougeL: Score(precision=0.6923076923076923, recall=0.6666666666666666, fmeasure=0.6792452830188679)


# PROBLEM 3

1. **Translation is tricky**: Translating from one language to another isn't always straightforward. The difficulty can change depending on the languages and the direction of translation (like from Telugu to English or English to Telugu).

2. **IndicTrans is better**: Between the two models tested, IndicTrans and NLLB, IndicTrans generally does a better job. It gets higher scores on both Bleu and Rouge metrics, which measure the quality of translations.And IndicTrans takes less time than nllb.

3. **Language structure matters**: Languages have different structures and vocabularies. So, translating between certain languages (like Telugu to English or Telugu to Hindi) can be more challenging. This is reflected in lower scores for these language pairs.

4. **Translation direction matters**: The quality of translation can also depend on the direction of translation. For instance, translating from English to Telugu generally gets higher scores than translating from Telugu to English.



**Bleu Scores**: The Bleu scores indicate the quality of translations, with higher scores indicating better translations. The IndicTrans model consistently outperforms the NLLB model in terms of Bleu scores for all language pairs.

**Rouge Scores**: Rouge scores (Rouge-1, Rouge-2, Rouge-L) measure the overlap between the generated translations and reference translations. Again, the IndicTrans model generally achieves higher Rouge scores compared to the NLLB model across all language pairs.


In a nutshell, IndicTrans is the better model for translating between Telugu, English, and Hindi. But remember, the specific languages and the direction of translation can affect the translation quality. So, it's important to consider these factors when choosing a translation model for a task.